# Parameter types

You can get some of the benefit of Param from Parameter and Parameterized alone, such as having constant or readonly parameters, parameter value inheritance, and parameter docstrings. However, you'll typically want a more specialized Parameter type so that you can enforce type and bounds restrictions and enable suitable behavior specialized for that type of parameter. Param ships with a large set of pre-defined more-specific Parameter types, and additional custom parameters can and should be added for any domain-specific parameter types needed.

The predefined types are organized into a class hierarchy where each type is a subclass of Parameter:

- [String](#strings): String value, optionally constrained by a regular expression
- [Color](#colors): A hex string specifying an RGB color, or a standard named color
- [Boolean](#booleans): True or False (or None, if allowed)
  * [Event](#invocations): True/False parameter used to trigger actions (see [Dependencies and watchers](Dependencies_and_Watchers.ipynb)).
- [Dynamic](#numbers): Base class for values that can be set to a callable that returns a concrete value
  * [Number](#numbers): Any type of numeric value
    - [Integer](#numbers): Positive or negative integer value
    - [Magnitude](#numbers): Positive value in the inclusive range 0.0,1.0
    - [Date](#numbers): Date or datetime type
    - [CalendarDate](#numbers): Date (not datetime)
- [Tuple](#tuples): Python tuple of a fixed length and optional fixed type
  * [NumericTuple](#tuples): Python tuple of a fixed length and a numeric type
    - [XYCoordinates](#tuples): Position in an x,y plane
    - [Range](#tuples): A numeric range or interval
      * [DateRange](#tuples): A range of dates or datetimes
      * [CalendarDateRange](#tuples): A range of dates (not datetimes)
- [List](#lists): A list of objects, potentially of a fixed, min, or max length
  * [HookList](#lists): A list of callables, for calling user-defined code at a processing stage
- [Path](#paths): A POSIX-style string specifying the location of a local file or folder
  * [Filename](#paths): A POSIX-style string specifying the location of a local file
  * [Foldername](#paths): A POSIX-style string specifying the location of a local folder
- [SelectorBase](#selectors): Abstract superclass covering various selector parameter types
  * [Selector](#selectors): One object selected out of a provided ordered list of objects
    - [FileSelector](#selectors): One filename selected out of those matching a provided glob
    - [ListSelector](#selectors): Multiple objects selected out of a provided list of objects
      - [MultiFileSelector](#selectors): Multiple filenames selected out of those matching a provided glob
  * [ClassSelector](#classSelectors): An instance or class of a specified Python class or superclass
    - [Dict](#classSelectors): A Python dictionary
    - [Array](#classSelectors): NumPy array
    - [Series](#classSelectors): A Pandas Series
    - [DataFrame](#classSelectors): A Pandas DataFrame
- [Callable](#invocations): A callable object, such as a function.
  * [Action](#invocations): A callable with no arguments, ready to invoke
- [Composite](#invocations): A list of other attributes or parameters of this class, settable and readable as a group

The full behavior of these types is covered in the [Reference Manual](https://param.holoviz.org/Reference_Manual/param.html). Here we will discuss the major categories of Parameter type and how to use them, including examples of what each type does _not_ allow (labeled `with param.exceptions_summarized():`).  Each of these classes is also suitable for subclassing to create more specialized types enforcing your own specific constraints.  After reading about Parameters in general, feel free to skip around in this page and only look at the Parameter types of interest to you!

## Strings

- `param.String`: String value, with optional regular expression (regex) constraints

A `param.String` may be set to any Python string value by default, or it may be constrained to match a provided regular expression `regex`. Like all other Parameters, it can optionally also `allow_None`, which will be true by default if the default value is None.

In [ ]:
import param

class S(param.Parameterized):
    s = param.String('Four score', regex='[A-Z][a-z][a-z][a-z ]*')

s = S()
s.s

In [ ]:
with param.exceptions_summarized():
    s.s = 5

In [ ]:
s.s = 'Forever after'

In [ ]:
with param.exceptions_summarized():
    s.s = 'four of spades'

In [ ]:
ip_regex = '^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$'
email_regex = '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

class I(param.Parameterized):
    ip_address = param.String('192.1.0.1', regex=ip_regex)
    email = param.String('example@me.com', regex=email_regex)
i = I()
i.ip_address

In [ ]:
i.ip_address="10.0.0.2"
i.email = "user@gmail.com"

In [ ]:
with param.exceptions_summarized():
    i.ip_address='192.x.1.x'

## Colors

- `param.Color`: Named color or hex RGB string (with or without a # prefix)


A Color parameter specifies one of the standard [web color names](https://www.w3.org/TR/css-color-3/#svg-color) or an arbitrary hex RGB string. To support only hex RGB strings, specify `allow_named=False`.

lemonchiffon

In [ ]:
class C(param.Parameterized):
    c = param.Color('#EEFF00')

c = C()
c.c

In [ ]:
c.c = 'lemonchiffon'
c.c

In [ ]:
with param.exceptions_summarized():
    c.c = 'puce'

In [ ]:
with param.exceptions_summarized():
    c.c = '#abcdefg'

## Booleans

- `param.Boolean`: A True or False value (or None, if allow_None is true)

A Boolean may be True or False. Like all other Parameters, it can optionally also `allow_None`, which will be true by default if the default value is None.

In [ ]:
class B(param.Parameterized):
    b = param.Boolean(True)
    n = param.Boolean(None)

b = B()
b.b

In [ ]:
with param.exceptions_summarized():
    b.b=1

In [ ]:
with param.exceptions_summarized():
    b.b=None

In [ ]:
b.n = True
b.n = None

## Numbers

- `param.Number`: Python floats, int, and bignum values.
- `param.Integer`: Integer values.
- `param.Magnitude`: Same as `param.Number(..., bounds=(0.0,1.0))`.
- `param.Date`: Date or datetime value of type `datetime.datetime`, `datetime.date`, or `numpy.datetime64`.
- `param.CalendarDate`: Date value of type `datetime.date`.

A Number is the most common type of Parameter. All Numbers in param are of class Dynamic, which allows them to be set not just to a single value but to a value that can repeatedly be drawn from a distribution or a sequence. (See [Dynamic Parameters](Dynamic_Parameters.ipynb) for more information about using these dynamic features, which will not be further discussed here.) Any Number has a default value (potentially None if allowed) and optional bounds.

There are two types of bounds: ``bounds`` and ``softbounds``.  ``bounds`` are hard bounds: the parameter must have a value within the specified range.  The default bounds are (None,None), meaning there are actually no hard bounds.  One or both bounds can be set by specifying a value (e.g. `bounds=(None,10)` means there is no lower bound, and an upper bound of 10). Bounds are inclusive by default, but exclusivity can be specified for each bound by setting inclusive_bounds (e.g. `inclusive_bounds=(True,False)` specifies an exclusive upper bound). 

When not being dynamically generated, `bounds` are checked whenever a Number is created or set. Using a default value outside the hard bounds, or one that is not numeric, results in an exception. When being dynamically generated, bounds are checked when the value of a Number is _requested_ (since it has no specific numeric value when first set). A generated value that is not numeric, or is outside the hard bounds, results in an exception. 

A separate set of ``softbounds`` is present to indicate the _typical_ range of the parameter, but these bounds are not enforced by Param. Setting the soft bounds allows a user to know what ranges of values are likely to be useful and allows a GUI to know what values to display on sliders for the Number; `softbounds` are thus suggestions or hints rather than enforced limits. 

Similarly, an optional ``step`` value can be provided to indicate the granularity of this parameter. As for `softbounds`, Param does not force values to conform to the provided step value, but (if provided) the step can be queried by user code and used for parameter sweeps (starting at the `softbounds` low and incrementing in value by `step` until the `softbounds` high), or by GUI code to determine steps on a settings dial.

Several convenience methods for working with bounds are provided:
- `get_soft_bounds()`: return the soft bounds (or hard bounds, if no soft bounds), for code that needs to know the typical range for this Parameter.
- `crop_to_bounds(val)`: crop the provided value to fit into the hard bounds.
- `set_in_bounds(obj,val)`: silently crop the given value into the legal range and set to the result, for building an API or user interface that accepts free-form input. 

Numbers support a `set_hook` that can be set to a function like `def logging_set_hook(obj,val): log_value(val) return val`, which will be called whenever the value is set.

Using Number parameters:

In [ ]:
import param

class N(param.Parameterized):
    n = param.Number(5.6, bounds=(0,None), softbounds=(None,50))
    i = param.Integer(5, bounds=(0,50))
    
a = N()
a.n=2
a.n

In [ ]:
N.param.n.set_in_bounds(a,-10)
a.n

In [ ]:
a.param.n.set_in_bounds(a,-5)
a.n

In [ ]:
a.param.n.set_in_bounds(a,75)
a.n

In [ ]:
a.param.n.get_soft_bounds()

In [ ]:
with param.exceptions_summarized():
    a.n = -5

In [ ]:
a.n = 500
a.n

In [ ]:
with param.exceptions_summarized():
    a.i=5.7

In [ ]:
import datetime

class D(param.Parameterized):
    d = param.CalendarDate(datetime.date(1900, 1, 1))
    t = param.Date(datetime.datetime.fromisoformat('2002-12-25T00:00'))

d = D()
d.d = datetime.date.fromisoformat('2000-01-01')
d.d

In [ ]:
with param.exceptions_summarized():
    d.d = 2022

In [ ]:
d.t = datetime.date(1900, 1, 1)
d.t

In [ ]:
with param.exceptions_summarized():
    d.d = datetime.datetime.fromisoformat('2002-12-25T00:00')

## Tuples

- `param.Tuple`: Python tuple of a fixed length.
- `param.NumericTuple`: Python tuple of a fixed length, with numeric values.
- `param.XYCoordinates`: Python pair (2-tuple) of numeric values. Same as `param.NumericTuple(..., length=2)`, but semantically representing a 2D coordinate in a plane (e.g. for drawing programs or GUIs)
- `param.Range`: `NumericTuple` representing a numeric range with optional bounds and softbounds.
- `param.DateRange`: `Range` where the numeric type is a date or datetime (using same date types as `param.Date`).
- `param.CalendarDateRange`: `Range` where the numeric type is a `datetime.date`. 

A tuple Parameter accepts a Python tuple for the value. Tuple parameters have a fixed length, typically set by the default value of the parameter but settable as the `length` if the default value is None. Only a tuple of the specified length will be accepted when a value is set.

There are many tuple types as listed above, accepting either any type, numeric types, datetimes, dates, etc. `Range` types support `bounds`, `softbounds`, `inclusive_bounds`, and `step` on the numeric values in the tuple, similar to [Number](#Numbers) types.

In [ ]:
class T(param.Parameterized):
    t = param.Range((-10,10), bounds=(-100,None), softbounds=(None,100))
b = T()
b.t

In [ ]:
b.t = (50.2,50.3)
b.t

In [ ]:
with param.exceptions_summarized():
    b.t = 5

In [ ]:
with param.exceptions_summarized():
    b.t = (5,5,5)

In [ ]:
with param.exceptions_summarized():
    b.t = (5,"5")

In [ ]:
with param.exceptions_summarized():
    b.t = (-500,500)

In [ ]:
import datetime
class D(param.Parameterized):
    d = param.CalendarDateRange((datetime.date.fromisoformat('1900-01-01'),
                                 datetime.date.fromisoformat('1910-12-31')))
c = D()
c.d

In [ ]:
with param.exceptions_summarized():
    c.d=(1905, 1907)

## Lists

- `param.List`: A Python list of objects, usually of a specified type.
- `param.HookList`: A list of callable objects, for executing user-defined code at some processing stage

List Parameters accept a Python list of objects. Typically the `item_type` will be specified for those objects, so that the rest of the code does not have to further check types when it refers to those values. Where appropriate, the `bounds` of the list can be set as (_min_length_, _max_length_), defaulting to `(0,None)`. Because List parameters already have an empty value ([]), they do not support `allow_None`.

A `param.HookList` is a list whose elements are callable objects (typically either functions or objects with a `__call__` method). A `HookList` is intended for providing user configurability at various stages of some processing algorithm or pipeline. At present, there is no validation that the provided callable takes any particular number or type of arguments.

In [ ]:
import param
class L(param.Parameterized):
    ls = param.List(["red","green","blue"], item_type=str, bounds=(0,10))

e = L()
e.ls

In [ ]:
with param.exceptions_summarized():
    e.ls = [1,2]

In [ ]:
with param.exceptions_summarized():
    e.ls = [str(i) for i in range(20)]

In [ ]:
class multi_stage_example(param.Parameterized):
    before = param.HookList()
    during = param.HookList()
    after  = param.HookList()
    
    values = param.List([1.5,-8.1,6.9,100.0], item_type=float)
    
    def __call__(self):
        for h in self.before: h(self)
        s = 0
        for v in self.values:
            v_ = v
            for h in self.during: v_ = h(v_)
            s += v_
        for h in self.after: h()
        return s

ex = multi_stage_example()
ex()

In [ ]:
def validate(obj):
    for i in obj.values:
        if i<0:
            print("Negative value found in argument")

m = multi_stage_example(before=[validate])

m()

In [ ]:
from math import fabs

ex.during=[abs]
ex()

## Paths

- `param.Path`: A POSIX-style string specifying the location of a local file or folder
- `param.Filename`: A POSIX-style string specifying the location of a local file
- `param.Foldername`: A POSIX-style string specifying the location of a local folder

A Path can be set to a string specifying the path of a file or folder. In code, the string should be specified in POSIX (UNIX) style, using forward slashes / and starting from / if absolute or some other character if relative. When retrieved, the string will be in the format of the user's operating system. 

Relative paths are converted to absolute paths by searching for a matching filename on the filesystem. If `search_paths` is provided and not empty, the folders in that list are searched for the given filename, in order, returning the absolute path for the first match found by appending the provided path to the search path. An IOError is raised if the file or folder is not found. If `search_paths` is empty (the default), the file or folder is expected to be in the current working directory.

Either a file or a folder name is accepted by `param.Path`, while `param.Filename` accepts only file names and `param.Foldername` accepts only folder names.

In [ ]:
class P(param.Parameterized):
    p = param.Path('Parameter_Types.ipynb')
    f = param.Filename('Parameter_Types.ipynb')
    d = param.Foldername('lib', search_paths=['/','/usr','/share'])
    
p = P()
p.p

In [ ]:
p.p='/usr/lib'
p.p

In [ ]:
p.f

In [ ]:
with param.exceptions_summarized():
    p.f='/usr/lib'

In [ ]:
p.d

In [ ]:
with param.exceptions_summarized():
    p.d='Parameter_Types.ipynb'

## Selectors

- `param.Selector`: One object selected out of a provided ordered list of objects
- `param.ListSelector`: Multiple objects selected out of a provided list of objects
- `param.FileSelector`: One filename selected out of those matching a provided glob
- `param.MultiFileSelector`: Multiple filenames selected out of those matching a provided glob

The value of a Selector is one or more items from a set of allowed values.  All Selector types must implement `get_range()`, providing a concrete list of available options for the value.

A `param.Selector` accepts a list or dictionary of `objects`, and has a single default (current) value that must be one of those objects. If not otherwise specified, the default will be the first item from the list or dictionary.  

Providing the objects as a list is appropriate for selecting among a set of strings, or among a set of Parameterized objects that each have a "name" parameter. That way, a UI that lets users select by string will have a suitable string available for each object to let the user make a choice between them.

Otherwise, the objects should be provided as a _name_:_value_ dictionary, where the string name will be stored for use in such a UI, but is not otherwise accessed by Param. The values from setting and getting the parameter are always the actual underlying object, not the string names. Because the string name will need to be looked up from the value if this parameter is used in a UI, all objects need to be hashable via the `param.hashable()` function, which accepts Python literals plus list and dictionary types (treating them like tuples).

If the list of available objects is not meant be exhaustive, you can specify `check_on_set=False` (which automatically applies if the initial list is empty). Objects will then be added to the `objects` list whenever they are set, including as the initial default. `check_on_set=False` can be useful when the predefined set of objects is not exhaustive, letting a user select from the existing list for convenience while also being able to supply any other suitable object they can construct. When `check_on_set=True`, the initial value (and all subsequent values) must be in the `objects` list.

Because `Selector` is usually used to allow selection from a list of existing (instantiated) objects, `instantiate` is False by default, but you can specify `instantiate=True` if you want each copy of this Parameter value to be independent of other instances and superclasses.

In cases where the objects in the list cannot be known when writing the Parameterized class but can be calculated at runtime, you can supply a callable (of no arguments) to `compute_default_fn`. If provided, this callable will be invoked, with the result used as the initial value for this Parameter.

A `param.ListSelector` works just the same as a regular Selector, but the value is a _list_ of valid objects from the available objects, rather than just one. Each item in the list is checked against the `objects`, and thus the current value is thus a _subset_ of the `objects`, rather than just one of the objects.

A `param.FileSelector` works like a regular Selector with the value being a filename and the `objects` being computed from files on a file system. The files are specified as a `path` [glob](https://docs.python.org/3/library/glob.html), and all filenames matching the glob are valid `objects` for the parameter.

A `param.MultiFileSelector` is the analog of ListSelector but for files, i.e., again supporting a path glob but allowing the user to select a list of filenames rather than a single filename. The default value in this case is _all_ of the matched files, not just the first one.

In [ ]:
colors = ["red","green","blue"]

class S(param.Parameterized):
    o = param.Selector(colors)
    ls = param.ListSelector(colors[0:2], objects=colors)
    
s = S()
s.o

In [ ]:
s.o = "green"
s.o

In [ ]:
with param.exceptions_summarized():
    s.o = "yellow"

In [ ]:
with param.exceptions_summarized():
    s.o = 42

In [ ]:
s.ls

In [ ]:
s.ls=['blue']
s.ls

In [ ]:
with param.exceptions_summarized():
    s.ls=['red','yellow']
    s.ls

In [ ]:
class F(param.Parameterized):
    f = param.FileSelector(path='/usr/share/*')
    fs = param.MultiFileSelector(path='/usr/share/*')
    
f = F()
f.f

In [ ]:
f.param.f.objects[0:3]

In [ ]:
f.fs = f.param.fs.objects[0:2]
f.fs

## ClassSelectors

- `param.ClassSelector`: An instance or class of a specified Python class or superclass
- `param.Dict`: A Python dictionary
- `param.Array`: NumPy array
- `param.Series`: A Pandas Series
- `param.DataFrame`: A Pandas DataFrame

A ClassSelector has a value that is either an instance or a subclass of a specified Python `class_`. By default, requires an instance of that class, but specifying `is_instance=False` means that a subclass must be selected instead. 

Like Selector types, all ClassSelector types implement `get_range()`, in this case providing an introspected list of all the concrete (not abstract) subclasses available for the given class. If you want a class to be treated as abstract so that it does not show up in such a list, you can have it declare `__abstract=True` as a class attribute. In a GUI, the range list allows a user to select a type of object they want to create, and they can then separately edit the new object's parameters (if any) to configure it appropriately.

In [ ]:
class C(param.Parameterized):
    e_instance = param.ClassSelector(class_=ArithmeticError, default=ZeroDivisionError("1/0"))
    e_class    = param.ClassSelector(class_=ArithmeticError, default=ZeroDivisionError, is_instance=False)
    
c = C(e_class=OverflowError)
c.e_class, c.e_instance

In [ ]:
c.param.e_instance.get_range()

In [ ]:
c.param.e_class.get_range()

In [ ]:
with param.exceptions_summarized():
    c.e_class = Exception

In [ ]:
with param.exceptions_summarized():
    c.e_instance = ArithmeticError

In [ ]:
c.e_instance = ArithmeticError()
c.e_instance

Various types of ClassSelector are provided for specific data types:

- `param.Dict`: `class_=dict`, accepting a Python dictionary
- `param.Array`: `class=numpy.ndarray`, accepting a NumPy array
- `param.Series`: `class_=pandas.Series`, a Pandas Series. Accepts constraints on the number of `rows`, either as an integer length (e.g. `rows=10`) or a range tuple `rows=(2,4)`).
- `param.DataFrame`: `class_=pandas.DataFrame`, a Pandas DataFrame. Accepts constraints on the number of `rows` (as for `param.Series`) or `columns` (with numerical or range values as for `rows` or as a list of column names (which must appear in that order) or as a set of column names (which can appear in any order)).

In [ ]:
import numpy as np, pandas as pd

class D(param.Parameterized):
    d = param.Dict(dict(a=1, b="not set", c=2.0))
    a = param.Array(np.array([1,-1]))
    s = param.Series(pd.Series([1,-1]))
    f = param.DataFrame(pd.DataFrame({'a':[1,2], 'b':[2,3], 'c':[4,5]}), rows=(2,None), columns=set(['a','b']))

d = D()
d.d = {5:np.nan}
d.d

In [ ]:
with param.exceptions_summarized():
    d.d=[("a",1)]

In [ ]:
df = pd.DataFrame({'b':[-2,-3], 'a':[-1,-2]})
d.f = df
d.f

In [ ]:
with param.exceptions_summarized():
    df = pd.DataFrame({'a':[-2,-3], 'c':[-1,-2]})
    d.f = df

In [ ]:
with param.exceptions_summarized():
    df = pd.DataFrame({'a':[-2], 'b':[-1]})
    d.f = df

### `classlist` and `param.descendents`

If you are building a GUI or some other mechanism allowing the user to choose a class or an instance of a specified class in a ClassSelector, you may want to construct a list of all subclasses or superclasses of the given class. To make it easy to traverse upwards or downwards in the inheritance hierarchy in this way, param provides the `classlist` and `param.descendents` functions. `classlist` provides a list of the superclasses of the provided object, including itself, in order from least to most specific:

In [ ]:
from param.parameterized import classlist

classlist(D)

As you can see, `D` is a type of `Parameterized`, and a `Parameterized` is a type of Python object. Conversely (and typically more usefully), `param.descendents` provides a list of the subclasses of the provided object, including itself:

In [ ]:
param.descendents(param.SelectorBase)

As you can see, there are many subtypes of SelectorBase. This list is calculated from whatever subtypes are currently defined in this Python session. If you derive an additional subtype or load code that defines an additional subtype, this list will get longer, so you need to make sure that all such code has been executed before letting the user make a selection of a subtype.

## Invocations

- `param.Callable`: A callable object, such as a function
- `param.Action`: A callable with no arguments, ready to invoke
- `param.Event`: Empty action, for use in triggering events for watchers
- `param.Composite`: Wrapper around other parameters, letting them be set as a tuple

Invocation parameters are a loose group of types that either contain an executable (callable) object, are invoked to execute some other code, or are set to change the value of some other parameter(s) or attribute(s).

A Callable may be set to any callable object, typically either a function or else an instance of a class that provides a `__call__` method. At present, there is no validation that the provided callable takes any particular number or type of arguments. Lambdas can be provided, but note that the resulting parameter value will no longer be picklable, so if you need to use pickling (`setstate` and `getstate`), be sure to use a named function instead.

An Action is the same as a Callable, but is expected to have no arguments. In a GUI an Action is typically mapped to a button whose name or label is the name of this parameter. 

An Event Parameter has a Boolean value but is primarily intended for triggering events on its watchers. See [Dependencies and Watchers](Dependencies_and_Watchers.ipynb) for the details.

A Composite Parameter has a value that is looked up from the value of a list of attributes of this class (which may or may not be parameters) and that when set changes the values of those other attributes or parameters. This type of Parameter can be useful for treating a set of related values as a group for setting purposes, but as individual parameters for code that reads from them. As of Param 1.10, Composite parameters have not been tested with watchers and dependencies and may not behave appropriately for such uses.

In [ ]:
def identity(x): return x
    
def print_time_of_day():
    print(datetime.date.today())

class A(param.Parameterized):
    transformer = param.Callable(identity)
    a = param.Action(print_time_of_day)
    
    def __call__(self, x):
        return self.transformer(x)
 
a = A()
a(5)

In [ ]:
def double(x):
    return 2*x
    
d = A(transformer=double)
d(5)

In [ ]:
d.a()

In [ ]:
with param.exceptions_summarized():
    d.a = 5